In [1]:
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id='c950cc1a56324bb7bd4e8de99ef42879',client_secret='0a89020467114989aa923125a45a7fba',))
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
spotify_df = pd.read_csv('tracks.csv')
artists_df = pd.read_csv('artists.csv')
spotify_df=spotify_df[['id', 'name', 'artists','id_artists', 
                    'popularity', 'danceability', 
                    'energy', 'loudness', 'speechiness', 
                    'acousticness', 'instrumentalness', 'liveness',
                    'valence', 'tempo']]

In [3]:
spotify_df.head()

,id,name,artists,id_artists,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,35iwgR4jXetI318WEWsa1Q,Carve,['Uli'],['45tIt06XoI0Iio4LBEVpls'],6,0.645,0.4450,-13.338,0.4510,0.674,0.7440,0.151,0.127,104.851
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],0,0.695,0.2630,-22.136,0.9570,0.797,0.0000,0.148,0.655,102.009
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],0,0.434,0.1770,-21.180,0.0512,0.994,0.0218,0.212,0.457,130.418
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],0,0.321,0.0946,-27.961,0.0504,0.995,0.9180,0.104,0.397,169.980
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],0,0.402,0.1580,-16.900,0.0390,0.989,0.1300,0.311,0.196,103.220


In [4]:
spotify_df.shape

(586672, 14)

In [5]:
artists_df['genres'].values[48]

"['carnaval cadiz']"

In [6]:
def string_to_list(list_str):
    return list_str.strip('][').split(', ')

In [7]:
def first_value(list_str):
    return list_str.split(',')[0][2:-2]

In [8]:
artists_df['genres_fix'] = artists_df['genres'].apply(string_to_list)

In [9]:
artists_df['genres_fix'].values[136][1]

"'harp'"

In [10]:
spotify_df['artists_fix'] = spotify_df['artists'].apply(string_to_list)
spotify_df['id_artists_fix'] = spotify_df['id_artists'].apply(first_value)

In [11]:
spotify_df.head()

,id,name,artists,id_artists,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artists_fix,id_artists_fix
0,35iwgR4jXetI318WEWsa1Q,Carve,['Uli'],['45tIt06XoI0Iio4LBEVpls'],6,0.645,0.4450,-13.338,0.4510,0.674,0.7440,0.151,0.127,104.851,['Uli'],45tIt06XoI0Iio4LBEVpls
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],0,0.695,0.2630,-22.136,0.9570,0.797,0.0000,0.148,0.655,102.009,['Fernando Pessoa'],14jtPCOoNZwquk5wd9DxrY
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],0,0.434,0.1770,-21.180,0.0512,0.994,0.0218,0.212,0.457,130.418,['Ignacio Corsini'],5LiOoJbxVSAMkBS2fUm3X2
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],0,0.321,0.0946,-27.961,0.0504,0.995,0.9180,0.104,0.397,169.980,['Ignacio Corsini'],5LiOoJbxVSAMkBS2fUm3X2
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],0,0.402,0.1580,-16.900,0.0390,0.989,0.1300,0.311,0.196,103.220,['Dick Haymes'],3BiJGZsyX9sJchTqcSA7Su


In [12]:
spotify_df['name_artists']=spotify_df.apply(lambda row: str(row['name']) + str(row['artists_fix'][0]), axis=1)

In [13]:
spotify_df[spotify_df['name']=='Hello'].head(5)

,id,name,artists,id_artists,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artists_fix,id_artists_fix,name_artists
56516,6HMvJcdw6qLsyV1b5x29sa,Hello,['Lionel Richie'],['3gMaNLQm7D9MornNILzdSl'],69,0.424,0.222,-15.053,0.0391,0.6460,0.000000,0.148,0.0606,61.648,['Lionel Richie'],3gMaNLQm7D9MornNILzdSl,Hello'Lionel Richie'
57128,1b16zIZIdL2LIMfDiANwIk,Hello,['Lionel Richie'],['3gMaNLQm7D9MornNILzdSl'],43,0.563,0.262,-13.498,0.0333,0.5160,0.000000,0.194,0.0626,124.206,['Lionel Richie'],3gMaNLQm7D9MornNILzdSl,Hello'Lionel Richie'
73926,60uesDF4UyLUs61FgiDkdp,Hello,"['Ice Cube', 'Dr. Dre', 'MC Ren']","['3Mcii5XWf6E0lrY3Uky4cA', '6DPYiyq5kWVQS4RGwx...",60,0.903,0.610,-6.176,0.3390,0.1550,0.000000,0.389,0.6270,91.460,"['Ice Cube', 'Dr. Dre', 'MC Ren']",3Mcii5XWf6E0lrY3Uky4c,Hello'Ice Cube'
76912,0aYUqsvZG7bAslrUkd9Z0g,Hello,['Evanescence'],['5nGIFgo0shDenQYSE0Sn7c'],60,0.416,0.178,-11.428,0.0310,0.9120,0.000138,0.342,0.0699,139.969,['Evanescence'],5nGIFgo0shDenQYSE0Sn7c,Hello'Evanescence'
87694,6BAnxKyld909yo6Pk1DO3r,Hello,['OMFG'],['0UdoKjWIe3tHyiqc4qT3Oz'],65,0.844,0.864,-4.483,0.0348,0.0339,0.216000,0.386,0.7960,104.997,['OMFG'],0UdoKjWIe3tHyiqc4qT3Oz,Hello'OMFG'


In [14]:
spotify_df.drop_duplicates('name_artists',inplace = True)

In [15]:
spotify_df[spotify_df['name']=='Hello'].head(5)

,id,name,artists,id_artists,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artists_fix,id_artists_fix,name_artists
56516,6HMvJcdw6qLsyV1b5x29sa,Hello,['Lionel Richie'],['3gMaNLQm7D9MornNILzdSl'],69,0.424,0.222,-15.053,0.0391,0.6460,0.000000,0.1480,0.0606,61.648,['Lionel Richie'],3gMaNLQm7D9MornNILzdSl,Hello'Lionel Richie'
73926,60uesDF4UyLUs61FgiDkdp,Hello,"['Ice Cube', 'Dr. Dre', 'MC Ren']","['3Mcii5XWf6E0lrY3Uky4cA', '6DPYiyq5kWVQS4RGwx...",60,0.903,0.610,-6.176,0.3390,0.1550,0.000000,0.3890,0.6270,91.460,"['Ice Cube', 'Dr. Dre', 'MC Ren']",3Mcii5XWf6E0lrY3Uky4c,Hello'Ice Cube'
76912,0aYUqsvZG7bAslrUkd9Z0g,Hello,['Evanescence'],['5nGIFgo0shDenQYSE0Sn7c'],60,0.416,0.178,-11.428,0.0310,0.9120,0.000138,0.3420,0.0699,139.969,['Evanescence'],5nGIFgo0shDenQYSE0Sn7c,Hello'Evanescence'
87694,6BAnxKyld909yo6Pk1DO3r,Hello,['OMFG'],['0UdoKjWIe3tHyiqc4qT3Oz'],65,0.844,0.864,-4.483,0.0348,0.0339,0.216000,0.3860,0.7960,104.997,['OMFG'],0UdoKjWIe3tHyiqc4qT3Oz,Hello'OMFG'
88047,1Yk0cQdMLx5RzzFTYwmuld,Hello,['Adele'],['4dpARuHxo51G3z768sgnrY'],77,0.585,0.409,-6.957,0.0293,0.3150,0.000000,0.0873,0.2840,78.983,['Adele'],4dpARuHxo51G3z768sgnrY,Hello'Adele'


In [16]:
artists_df.head()

,id,followers,genres,name,popularity,genres_fix
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0,[]
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0,[]
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0,[]
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0,[]
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0,[]


In [17]:
spotify_df_genres = spotify_df.merge(artists_df[['id', 'genres_fix']], left_on='id_artists_fix', right_on='id')

In [18]:
spotify_df_genres.head()

,id_x,name,artists,id_artists,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artists_fix,id_artists_fix,name_artists,id_y,genres_fix
0,35iwgR4jXetI318WEWsa1Q,Carve,['Uli'],['45tIt06XoI0Iio4LBEVpls'],6,0.645,0.44500,-13.338,0.4510,0.674,0.744,0.1510,0.127,104.851,['Uli'],45tIt06XoI0Iio4LBEVpls,Carve'Uli',45tIt06XoI0Iio4LBEVpls,[]
1,0PH9AACae1f957JAavhOl2,Lazy Boi,['Uli'],['45tIt06XoI0Iio4LBEVpls'],0,0.298,0.46000,-18.645,0.4530,0.521,0.856,0.4360,0.402,87.921,['Uli'],45tIt06XoI0Iio4LBEVpls,Lazy Boi'Uli',45tIt06XoI0Iio4LBEVpls,[]
2,2SiNuAZ6jIU9xhClRKXcST,Sketch,['Uli'],['45tIt06XoI0Iio4LBEVpls'],0,0.634,0.00399,-29.973,0.0377,0.926,0.919,0.1050,0.396,79.895,['Uli'],45tIt06XoI0Iio4LBEVpls,Sketch'Uli',45tIt06XoI0Iio4LBEVpls,[]
3,4vV7uBcF2AnjNTOejBS5oL,L'enfer,['Uli'],['45tIt06XoI0Iio4LBEVpls'],0,0.657,0.32500,-14.319,0.2540,0.199,0.856,0.0931,0.105,81.944,['Uli'],45tIt06XoI0Iio4LBEVpls,L'enfer'Uli',45tIt06XoI0Iio4LBEVpls,[]
4,598LlBn6jpEpVbLjmZPsYV,Graphite,['Uli'],['45tIt06XoI0Iio4LBEVpls'],0,0.644,0.68400,-8.247,0.1990,0.144,0.802,0.0847,0.138,100.031,['Uli'],45tIt06XoI0Iio4LBEVpls,Graphite'Uli',45tIt06XoI0Iio4LBEVpls,[]


In [19]:
spotify_df_genres['id_artists'] = spotify_df_genres['id_artists_fix']
spotify_df_genres['artists'] = spotify_df_genres['artists_fix']
spotify_df_genres = spotify_df_genres.drop(columns=['id_artists_fix','artists_fix','name_artists','id_y'])

In [20]:
spotify_df_genres['genres_fix'] = spotify_df_genres['genres_fix'].apply(lambda d: d if isinstance(d, list) else [])

In [21]:
#https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list
mlb = MultiLabelBinarizer(sparse_output=True)

spotify_df_genres = spotify_df_genres.join(pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(spotify_df_genres.pop('genres_fix')),
                index=spotify_df_genres.index,
                columns=mlb.classes_))

In [22]:
spotify_df_genres.shape

(419665, 4421)

In [23]:
df_clean = spotify_df_genres.copy()
df_clean['popularity'] = spotify_df_genres['popularity']/100
#normalize float variables
df_clean['loudness'] = (spotify_df_genres['loudness'] - min(spotify_df_genres['loudness'])) / (max(spotify_df_genres['loudness'])-min(spotify_df_genres['loudness']))
df_clean['tempo'] = (spotify_df_genres['tempo']-min(spotify_df_genres['tempo']))/(max(spotify_df_genres['tempo'])-min(spotify_df_genres['tempo']))
df_clean['speechiness'] = (spotify_df_genres['speechiness']-min(spotify_df_genres['speechiness']))/(max(spotify_df_genres['speechiness'])-min(spotify_df_genres['speechiness']))

In [24]:
user_df = pd.DataFrame(columns=df_clean.columns).drop(columns=['id_artists'])


In [25]:
user_df.head()

,id_x,name,artists,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,...,'zikir','zillertal','zim urban groove','zither','zolo','zouglou','zouk riddim','zouk','zurich indie','zydeco'


In [26]:
#EE 'Spotify','37i9dQZF1DX9tPFwDMOaN1'
results = sp.user_playlist_tracks('Spotify','37i9dQZF1DX9tPFwDMOaN1')
tracks = results['items']
while results['next']:
    results = sp.next(results)
    tracks.extend(results['items'])
for i in range(len(tracks)):
    playlist_tracks_id = tracks[i]['track']['id']
    features = sp.audio_features(playlist_tracks_id)
    artist_uri = tracks[i]["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    artist_genres = artist_info["genres"]
    song_row = pd.DataFrame.from_records([{ 'id_x': tracks[i]['track']['id'],
                                                         'name': tracks[i]['track']['name'],
                                                         'artists':tracks[i]['track']['artists'][0]['name'],
                                                         'popularity':tracks[i]['track']['popularity'],
                                                         'danceability':features[0]['danceability'],
                                                         'energy':features[0]['energy'],
                                                         'loudness':features[0]['loudness'],
                                                         'speechiness':features[0]['speechiness'],
                                                         'acousticness':features[0]['acousticness'],
                                                         'instrumentalness':features[0]['instrumentalness'],
                                                         'liveness':features[0]['liveness'],
                                                         'valence':features[0]['valence'],
                                                         'tempo':features[0]['tempo']
                                                         }])
    for genres in artist_genres:
        fixed_genres = repr(genres)
        if fixed_genres not in df_clean.columns:
            df_clean[fixed_genres] = 0
        song_row[fixed_genres] = 1
    user_df = pd.concat([user_df, song_row])
    
user_df = user_df.fillna(0)

In [27]:
user_df

,id_x,name,artists,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,...,'zikir','zillertal','zim urban groove','zither','zolo','zouglou','zouk riddim','zouk','zurich indie','zydeco'
0,2gYj9lubBorOPIVWsTXugG,After LIKE,IVE,86,0.680,0.922,-1.215,0.1210,0.10300,0.000000,...,0,0,0,0,0,0,0,0,0,0
0,0RDqNCRBGrSegk16Avfzuq,Talk that Talk,TWICE,84,0.772,0.907,-2.438,0.1240,0.13600,0.000000,...,0,0,0,0,0,0,0,0,0,0
0,0a4MMyCrzT0En247IhqZbD,Hype Boy,NewJeans,86,0.593,0.935,-3.430,0.2250,0.26800,0.000000,...,0,0,0,0,0,0,0,0,0,0
0,0skYUMpS0AcbpjcGsAbRGj,Pink Venom,BLACKPINK,91,0.798,0.697,-7.139,0.0891,0.02020,0.000000,...,0,0,0,0,0,0,0,0,0,0
0,1oen3GpTcA486fTHaT7neg,FOREVER 1,Girls' Generation,81,0.615,0.871,-1.248,0.0516,0.04270,0.000000,...,0,0,0,0,0,0,0,0,0,0
0,2pIUpMhHL6L9Z5lnKxJJr9,Attention,NewJeans,86,0.811,0.648,-3.684,0.0368,0.23600,0.000000,...,0,0,0,0,0,0,0,0,0,0
0,7nj5G4aPUJD0TnNF6SqcrX,FOCUS,HA SUNG WOON,60,0.887,0.650,-5.427,0.0779,0.00200,0.000000,...,0,0,0,0,0,0,0,0,0,0
0,2WoluqyWzsgRmFCeHeGlnm,SNEAKERS,ITZY,79,0.935,0.849,-3.047,0.0779,0.01920,0.000000,...,0,0,0,0,0,0,0,0,0,0
0,3lOMJTQTd6J34faYwASc33,POP!,NAYEON,85,0.795,0.859,-2.994,0.0542,0.03610,0.000000,...,0,0,0,0,0,0,0,0,0,0
0,52uklJhyhJbLvHrgkiqCaW,WHISPER,THE BOYZ,68,0.652,0.924,-2.502,0.0562,0.00242,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [28]:
df_clean

,id_x,name,artists,id_artists,popularity,danceability,energy,loudness,speechiness,acousticness,...,'zikir','zillertal','zim urban groove','zither','zolo','zouglou','zouk riddim','zouk','zurich indie','zydeco'
0,35iwgR4jXetI318WEWsa1Q,Carve,['Uli'],45tIt06XoI0Iio4LBEVpls,0.06,0.645,0.44500,0.713748,0.465428,0.674,...,0,0,0,0,0,0,0,0,0,0
1,0PH9AACae1f957JAavhOl2,Lazy Boi,['Uli'],45tIt06XoI0Iio4LBEVpls,0.00,0.298,0.46000,0.632572,0.467492,0.521,...,0,0,0,0,0,0,0,0,0,0
2,2SiNuAZ6jIU9xhClRKXcST,Sketch,['Uli'],45tIt06XoI0Iio4LBEVpls,0.00,0.634,0.00399,0.459297,0.038906,0.926,...,0,0,0,0,0,0,0,0,0,0
3,4vV7uBcF2AnjNTOejBS5oL,L'enfer,['Uli'],45tIt06XoI0Iio4LBEVpls,0.00,0.657,0.32500,0.698743,0.262126,0.199,...,0,0,0,0,0,0,0,0,0,0
4,598LlBn6jpEpVbLjmZPsYV,Graphite,['Uli'],45tIt06XoI0Iio4LBEVpls,0.00,0.644,0.68400,0.791621,0.205366,0.144,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419660,0MmaEacabpK8Yp3Mdeo5uY,下雨天,['芝麻'],5VGgFE9nPgMfEnYiPT5J2B,0.50,0.528,0.67300,0.862105,0.032405,0.143,...,0,0,0,0,0,0,0,0,0,0
419661,1dKxf4Ht2SsKLyXfSDJAgy,The Cutest Puppy,['Laureen Conrad'],7vgGpuiXdNlCmc994PlMlz,0.67,0.609,0.01720,0.480712,0.121775,0.996,...,0,0,0,0,0,0,0,0,0,0
419662,0SjsIzJkZfDU7wlcdklEFR,John Brown's Song,['Gregory Oberle'],4MxqhahGRT4BPz1PilXGeu,0.66,0.562,0.03310,0.526936,0.106295,0.996,...,0,0,0,0,0,0,0,0,0,0
419663,5rgu12WBIHQtvej2MdHSH0,云与海,['阿YueYue'],1QLBXKM5GCpyQQSVMNZqrZ,0.50,0.560,0.51800,0.803491,0.030134,0.785,...,0,0,0,0,0,0,0,0,0,0


In [29]:
user_df_clean = user_df.copy()
user_df_clean['popularity'] = user_df['popularity']/100
user_df_clean['loudness'] = (user_df['loudness'] - min(spotify_df['loudness'])) / (max(spotify_df['loudness'])-min(spotify_df['loudness']))
user_df_clean['tempo'] = (user_df['tempo']-min(spotify_df['tempo']))/(max(spotify_df['tempo'])-min(spotify_df['tempo']))
user_df_clean.head(8)

,id_x,name,artists,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,...,'zikir','zillertal','zim urban groove','zither','zolo','zouglou','zouk riddim','zouk','zurich indie','zydeco'
0,2gYj9lubBorOPIVWsTXugG,After LIKE,IVE,0.86,0.680,0.922,0.899183,0.1210,0.1030,0.0,...,0,0,0,0,0,0,0,0,0,0
0,0RDqNCRBGrSegk16Avfzuq,Talk that Talk,TWICE,0.84,0.772,0.907,0.880476,0.1240,0.1360,0.0,...,0,0,0,0,0,0,0,0,0,0
0,0a4MMyCrzT0En247IhqZbD,Hype Boy,NewJeans,0.86,0.593,0.935,0.865302,0.2250,0.2680,0.0,...,0,0,0,0,0,0,0,0,0,0
0,0skYUMpS0AcbpjcGsAbRGj,Pink Venom,BLACKPINK,0.91,0.798,0.697,0.808569,0.0891,0.0202,0.0,...,0,0,0,0,0,0,0,0,0,0
0,1oen3GpTcA486fTHaT7neg,FOREVER 1,Girls' Generation,0.81,0.615,0.871,0.898678,0.0516,0.0427,0.0,...,0,0,0,0,0,0,0,0,0,0
0,2pIUpMhHL6L9Z5lnKxJJr9,Attention,NewJeans,0.86,0.811,0.648,0.861417,0.0368,0.2360,0.0,...,0,0,0,0,0,0,0,0,0,0
0,7nj5G4aPUJD0TnNF6SqcrX,FOCUS,HA SUNG WOON,0.60,0.887,0.650,0.834756,0.0779,0.0020,0.0,...,0,0,0,0,0,0,0,0,0,0
0,2WoluqyWzsgRmFCeHeGlnm,SNEAKERS,ITZY,0.79,0.935,0.849,0.871161,0.0779,0.0192,0.0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
playlist_vector = user_df_clean.copy().drop(columns=['id_x', 'name', 'artists'])
playlist_vector = playlist_vector[(playlist_vector.columns)].mean()
print(playlist_vector)

popularity        0.760200
danceability      0.705600
energy            0.790200
loudness          0.858192
speechiness       0.118386
                    ...   
'zouglou'         0.000000
'zouk riddim'     0.000000
'zouk'            0.000000
'zurich indie'    0.000000
'zydeco'          0.000000
Length: 4417, dtype: float64


In [31]:
def cos_sim_recommender(df, features):
    df['cosine_sim'] = cosine_similarity(df.drop(['id_x', 'name','artists','id_artists'], axis = 1).values, features.values.reshape(1, -1))[:,0]
    playlist_df_top_20 = df.sort_values('cosine_sim',ascending = False).head(20)
    return playlist_df_top_20

In [32]:
cos_sim_recommendations = cos_sim_recommender(df_clean,playlist_vector)

In [33]:
cos_sim_recommendations

,id_x,name,artists,id_artists,popularity,danceability,energy,loudness,speechiness,acousticness,...,'zillertal','zim urban groove','zither','zolo','zouglou','zouk riddim','zouk','zurich indie','zydeco',cosine_sim
226101,5xrtzzzikpG3BLbo4q1Yul,LILAC,['IU'],3HqSLMAZ3g3d5poNaI7GOU,0.80,0.738,0.890,0.882969,0.077193,0.08950,...,0,0,0,0,0,0,0,0,0,0.972091
226102,4RewTiGEGoO7FWNZUmp1f4,Celebrity,['IU'],3HqSLMAZ3g3d5poNaI7GOU,0.79,0.730,0.784,0.877692,0.071311,0.19100,...,0,0,0,0,0,0,0,0,0,0.968534
252352,5iIpbD34k4wnuRMZDNnuWf,I'm Not Cool,['HyunA'],3UwlejyX2b458azZ7eCnHb,0.75,0.883,0.845,0.886120,0.060372,0.01610,...,0,0,0,0,0,0,0,0,0,0.968440
226134,7CZRguMolNqIobnXxpV735,Coin,['IU'],3HqSLMAZ3g3d5poNaI7GOU,0.73,0.725,0.879,0.866511,0.104231,0.00849,...,0,0,0,0,0,0,0,0,0,0.967936
380954,4Dq2FjRkOTHcssggMdYK8J,MY TREASURE,['TREASURE'],3KonOYiLsU53m4yT7gNotP,0.69,0.730,0.785,0.847956,0.080495,0.11100,...,0,0,0,0,0,0,0,0,0,0.966748
239104,52CBUrIdyf8tbZaUY9iawE,After School,['Weeekly'],73B9bjqS2Z5KLXNGqXf64m,0.78,0.634,0.928,0.893157,0.053044,0.00719,...,0,0,0,0,0,0,0,0,0,0.966589
252365,7wDVvxMUdW5MtJUqFtuXUz,Bicycle,['CHUNG HA'],2PSJ6YriU7JsFucxACpU7Y,0.74,0.625,0.903,0.879237,0.061300,0.00600,...,0,0,0,0,0,0,0,0,0,0.965836
253339,0jFHMDRXxKaREor3hBEEST,Gashina,['SUNMI'],6MoXcK2GyGg7FIyxPU5yW6,0.69,0.722,0.833,0.857884,0.097007,0.05560,...,0,0,0,0,0,0,0,0,0,0.965524
226103,4Dr2hJ3EnVh2Aaot6fRwDO,Blueming,['IU'],3HqSLMAZ3g3d5poNaI7GOU,0.72,0.819,0.674,0.869662,0.060578,0.08490,...,0,0,0,0,0,0,0,0,0,0.964610
252372,6kKC35sKUh2FUx4M0qAL44,Candy,['BAEKHYUN'],4ufh0WuMZh6y4Dmdnklvdl,0.71,0.649,0.767,0.859061,0.046956,0.24000,...,0,0,0,0,0,0,0,0,0,0.964202


In [34]:
def euclidean_recommender(df, features):
    df['euclidean_dist'] = np.linalg.norm(df.drop(['id_x', 'name','artists','id_artists','cosine_sim'], axis = 1).sub(features.squeeze(), axis=1),axis=1)
    playlist_df_top_20 = df.sort_values('euclidean_dist',ascending = True).head(20)
    return playlist_df_top_20

In [35]:
euclidean_recommendations = euclidean_recommender(df_clean,playlist_vector)

In [36]:
euclidean_recommendations

,id_x,name,artists,id_artists,popularity,danceability,energy,loudness,speechiness,acousticness,...,'zim urban groove','zither','zolo','zouglou','zouk riddim','zouk','zurich indie','zydeco',cosine_sim,euclidean_dist
380954,4Dq2FjRkOTHcssggMdYK8J,MY TREASURE,['TREASURE'],3KonOYiLsU53m4yT7gNotP,0.69,0.730,0.785,0.847956,0.080495,0.11100,...,0,0,0,0,0,0,0,0,0.966748,0.504069
226102,4RewTiGEGoO7FWNZUmp1f4,Celebrity,['IU'],3HqSLMAZ3g3d5poNaI7GOU,0.79,0.730,0.784,0.877692,0.071311,0.19100,...,0,0,0,0,0,0,0,0,0.968534,0.512502
253339,0jFHMDRXxKaREor3hBEEST,Gashina,['SUNMI'],6MoXcK2GyGg7FIyxPU5yW6,0.69,0.722,0.833,0.857884,0.097007,0.05560,...,0,0,0,0,0,0,0,0,0.965524,0.515641
335888,1cgBWgoL6520lR2QZDzdGN,MOVE,['TAEMIN'],13rF01aOogvnkuQXOlgTW8,0.67,0.697,0.713,0.837111,0.118679,0.01530,...,0,0,0,0,0,0,0,0,0.964127,0.520604
226101,5xrtzzzikpG3BLbo4q1Yul,LILAC,['IU'],3HqSLMAZ3g3d5poNaI7GOU,0.80,0.738,0.890,0.882969,0.077193,0.08950,...,0,0,0,0,0,0,0,0,0.972091,0.521991
236254,2qPQ2MoVwf7VsKddqPfttw,Love Letter,['BOL4'],4k5fFEYgkWYrYvtOK3zVBl,0.60,0.728,0.764,0.879497,0.035604,0.15200,...,0,0,0,0,0,0,0,0,0.964000,0.525102
226099,1SwZVVH9BnXtLRLi2cbFhw,Good day,['IU'],3HqSLMAZ3g3d5poNaI7GOU,0.63,0.637,0.882,0.871360,0.067492,0.16500,...,0,0,0,0,0,0,0,0,0.963878,0.530447
226103,4Dr2hJ3EnVh2Aaot6fRwDO,Blueming,['IU'],3HqSLMAZ3g3d5poNaI7GOU,0.72,0.819,0.674,0.869662,0.060578,0.08490,...,0,0,0,0,0,0,0,0,0.964610,0.531659
252372,6kKC35sKUh2FUx4M0qAL44,Candy,['BAEKHYUN'],4ufh0WuMZh6y4Dmdnklvdl,0.71,0.649,0.767,0.859061,0.046956,0.24000,...,0,0,0,0,0,0,0,0,0.964202,0.534300
252367,0LweQRsfJ3pRAJJFy6DrR1,What Type of X,['Jessi'],64k5e9kV9MdukXjFrR5R37,0.75,0.776,0.692,0.859123,0.057276,0.00750,...,0,0,0,0,0,0,0,0,0.962412,0.535185
